Modules

In [1]:
import torch
import torch.nn.functional as F
import numpy as np

In [53]:
epochs = 1000
eval_iters = 250
learning_rate = .001
dropout_rate = .2

Dataset

In [3]:
file = open(r'dataset\wizard_of_oz.txt','r',encoding='utf-8')
text = file.read()
file.close()
len(text)

252051

In [4]:
tokens = sorted(set(text))

In [5]:
vocabulary = {token:index for index,token in enumerate(tokens)}
vocabulary_index = {index:token for index,token in enumerate(tokens)}

In [6]:
def vocabulary_encoder(sequence):
    return [vocabulary[s] for s in sequence]

def vocabulary_decoder(sequence):
    return ''.join([vocabulary_index[s] for s in sequence])


Pre-Processing

In [7]:
data = torch.tensor(vocabulary_encoder(text),dtype=torch.long)

In [8]:
n = int(.8*len(data))
train_data = data[:n]
test_data = data[n:]
block_size, batch_size = 8, 4

In [9]:
def get_batch(dataset):
    ix = torch.randint(len(dataset)-block_size, (batch_size,))
    x = torch.stack([dataset[i:i+block_size] for i in ix])
    y = torch.stack([dataset[i+1:i+block_size+1] for i in ix])
    # x, y = x.to(device)
    return x, y

In [241]:
x, y = get_batch(train_data)

In [150]:
print('input: ', x)
print('labels: ', y)

input:  tensor([[ 1, 64, 75, 78, 63, 63,  1, 77, 72, 71],
        [58, 71, 61,  1, 80, 58, 77, 62, 75, 62],
        [62,  1, 65, 62, 75, 62, 28,  3,  1, 58],
        [ 3, 48, 65, 58, 77,  1, 66, 76,  1, 75],
        [77, 65, 62,  1, 76, 78, 60, 60, 62, 76],
        [62, 75,  1, 75, 72, 80,  1, 72, 63,  1],
        [ 1, 64, 66, 75, 69, 14,  0,  0,  3, 44],
        [ 1, 59, 58, 61, 69, 82,  1, 63, 75, 66],
        [76,  1, 77, 72,  1, 60, 75, 58, 60, 68],
        [ 1, 58,  1, 69, 58, 71, 61, 66, 71, 64]])
labels:  tensor([[64, 75, 78, 63, 63,  1, 77, 72, 71],
        [71, 61,  1, 80, 58, 77, 62, 75, 62],
        [ 1, 65, 62, 75, 62, 28,  3,  1, 58],
        [48, 65, 58, 77,  1, 66, 76,  1, 75],
        [65, 62,  1, 76, 78, 60, 60, 62, 76],
        [75,  1, 75, 72, 80,  1, 72, 63,  1],
        [64, 66, 75, 69, 14,  0,  0,  3, 44],
        [59, 58, 61, 69, 82,  1, 63, 75, 66],
        [ 1, 77, 72,  1, 60, 75, 58, 60, 68],
        [58,  1, 69, 58, 71, 61, 66, 71, 64]])


Model

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
vocab_size = len(vocabulary)

In [54]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            if split=='train': X, Y = get_batch(train_data)
            else: X, Y = get_batch(test_data)

            with torch.no_grad():
                logits, loss = model(X, Y)
                losses[k] = loss.item()

        model.train()
        out[split] = losses.mean()

    return out

In [55]:
class Model(torch.nn.Module):
    def __init__(self,vocab_size,embedding_dim):
        super(Model,self).__init__()
        self.token_embedding_layer = torch.nn.Embedding(vocab_size,embedding_dim)

    def forward(self,input,targets=None):
        logits = self.token_embedding_layer(input)
        if targets == None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits,loss
    
    def generate(self, input, max_new_tokens):
        for i in range(max_new_tokens):
            logits, loss = self.forward(input)
            logits = logits[:,-1,:]
            probabilities = F.softmax(logits,dim=-1)
            input_next = torch.multinomial(probabilities,num_samples=1)
            input = torch.cat((input,input_next),dim=1)
        return input


In [56]:
model = Model(vocab_size, vocab_size)
model = model.to(device)
print(model)

Model(
  (token_embedding_layer): Embedding(92, 92)
)


In [57]:
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

In [58]:
for epoch in range(epochs):
    if epoch%100 == 0:
        losses = estimate_loss()
        print(f'step: {epoch}, loss: {losses}')
    xb, yb = get_batch(train_data)
    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss)

step: 0, loss: {'train': tensor(4.9999), 'val': tensor(4.9975)}
step: 100, loss: {'train': tensor(4.8727), 'val': tensor(4.8969)}
step: 200, loss: {'train': tensor(4.8069), 'val': tensor(4.8178)}
step: 300, loss: {'train': tensor(4.7206), 'val': tensor(4.7593)}
step: 400, loss: {'train': tensor(4.6307), 'val': tensor(4.6548)}
step: 500, loss: {'train': tensor(4.5660), 'val': tensor(4.5706)}
step: 600, loss: {'train': tensor(4.4747), 'val': tensor(4.5037)}
step: 700, loss: {'train': tensor(4.4157), 'val': tensor(4.4507)}
step: 800, loss: {'train': tensor(4.3308), 'val': tensor(4.3828)}
step: 900, loss: {'train': tensor(4.2689), 'val': tensor(4.3070)}
tensor(4.4643, grad_fn=<NllLossBackward0>)


In [34]:
context =  torch.tensor([vocabulary_encoder('Hello, how are you? my name is wiz')])
context

tensor([[36, 62, 69, 69, 72, 12,  1, 65, 72, 80,  1, 58, 75, 62,  1, 82, 72, 78,
         28,  1, 70, 82,  1, 71, 58, 70, 62,  1, 66, 76,  1, 80, 66, 83]])

In [35]:
generated_chars = vocabulary_decoder(model.generate(context, max_new_tokens=50)[0].tolist())
print(generated_chars)

Hello, how are you? my name is wizR3
Pt7BQWAU[Ipe.lG)*;N8MKiZmWrdDlS !AGu?3 E”*dTpNc


____

GPT

In [14]:
import torch
from torch.nn import functional as F

In [15]:
block_size = 8
batch_size = 4
epochs = 1000
learning_rate = .001
n_embd = 384
n_layer = 4

In [11]:
@torch.no_grad()
def calculate_loss():
    model.eval()
    losses = torch.zeros(250)
    for epoch in range(250):
        x, y = get_batch(test_data)
        logits, loss = model(x, y)
        losses[epoch] = loss.item()
    output = losses.mean()
    return output

In [ ]:
class GPTLanguageModel(torch.nn.Model):
    def __init__(self, vocab_size):
        self.token_embedding_table = torch.nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = torch.nn.Embedding(block_size, n_embd)
        self.blocks = torch.nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = torch.nn.LayerNorm(n_embd)
        self.lm_head = torch.nn.linear(n_embd, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None: loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)            
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)